In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [9]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    
    batch_size = 32
    target_size=(200,200)

    gen = ImageDataGenerator(rescale=1/255.)
    test_gen = gen.flow_from_directory('./data/test/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=False)

    model_list = ['EffNetV2M_17class_0004.h5', 'InceptionResNetV2_17class_0005.h5', 'VGG16_17class_0002.h5', 'Xception_17class_0004.h5']
    
    model = []
    acc = []
    pred = []
    
    for i, mod in enumerate(model_list):
        print('{}를 load 하고 있습니다.'.format(mod))
        model.append(tf.keras.models.load_model('./save_models/{}'.format(mod)))
        print('{} load 완료.'.format(mod))
        
        print('{} model의 accuracy를 평가중입니다.'.format(mod))
        acc.append(model[i].evaluate(test_gen)[1]) # accuracy 저장 
        print('{} model accuracy 평가완료.'.format(mod))
        
        print('{} model의 predict를 진행중입니다.'.format(mod))
        pred.append(model[i].predict(test_gen)) # predict list 저장
        print('{} model predict 완료.\n'.format(model_list[i]))
        
    w_pred = np.zeros_like(pred[0]) #weighted pred 저장
    for i in range(len(model)):
        w_pred += (pred[i]*acc[i])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Found 17340 images belonging to 17 classes.
EffNetV2M_17class_0004.h5를 load 하고 있습니다.
EffNetV2M_17class_0004.h5 load 완료.
EffNetV2M_17class_0004.h5 model의 accuracy를 평가중입니다.
542/542 [==============================] - 42s 77ms/step - loss: 0.5318 - accuracy: 0.8888
EffNetV2M_17class_0004.h5 model accuracy 평가완료.
EffNetV2M_17class_0004.h5 model의 predict를 진행중입니다.
EffNetV2M_17class_0004.h5 model predict 완료.

InceptionResNetV2_17class_0005.h5를 load 하고 있습니다.
InceptionResNetV2_17class_0005.h5 load 완료.
InceptionResNetV2_17class_0005.h5 model의 accuracy를 평가중입니다.
542/542 [==============================] - 37s 69ms/step - loss: 0.8966 - accuracy: 0.8242
InceptionResNetV2_17class_0005.h5 model accuracy 평가완료.
InceptionResNetV2_17class_0005.h5 model의 predict를 진행중입니다.
InceptionResNetV2_17class_0005.h5 model predict 완료.

VGG16_17class_0002.h5를 load 하고 있습니다.
VG

In [10]:
#test gen의 image와 label 추출
n_img = test_gen.n
steps = n_img//batch_size
print(f'n_img : {n_img} \t steps : {steps}')

imgs, labels = [], []
for i in range(steps):
    a, b = test_gen.next()
    imgs.extend(a)
    labels.extend(b)

imgs = np.asarray(imgs)
labels = np.asarray(labels)

n_img : 17340 	 steps : 541


In [106]:
#난수번째 class 예측
k = np.random.randint(0,n_img-1)
print(f'{k+1}번째 사진 class 예측\n')
print(*[int(i) for i in pred[0][k]*100], '\tacc:',round(acc[0]*100,2),'%')
print(*[int(i) for i in pred[1][k]*100], '\tacc:',round(acc[1]*100,2),'%')
print(*[int(i) for i in pred[2][k]*100], '\tacc:',round(acc[2]*100,2),'%')
print(*[int(i) for i in pred[3][k]*100], '\tacc:',round(acc[3]*100,2),'%')
print('\nEnsemble')
print(*[int(i) for i in w_pred[k]*100])
print('Ground Truth')
print(*[int(i) for i in labels[k]*100])

9905번째 사진 class 예측

0 0 70 0 16 0 0 0 0 0 3 0 0 8 0 0 0 	acc: 88.88 %
0 0 0 0 0 0 0 0 0 63 0 24 0 8 0 0 1 	acc: 82.42 %
0 0 0 0 0 0 0 0 0 96 0 0 0 3 0 0 0 	acc: 77.24 %
0 0 0 0 0 0 87 0 0 0 1 7 0 0 0 0 1 	acc: 86.04 %

Ensemble
0 0 62 1 14 0 75 0 0 126 4 26 0 17 0 0 2
Ground Truth
0 0 0 0 0 0 0 0 0 100 0 0 0 0 0 0 0


In [96]:
true=0
for i in range(len(imgs)):
    if np.argmax(w_pred[i])==np.argmax(labels[i]):
        true+=1
        
print('Ensemble accuracy',round(true/n_img*100,2),'%')

Ensemble accuracy 91.74 %
